In [1]:
!pip install git+https://github.com/bitbankinc/python-bitbankcc@fba9f83\#egg=python-bitbankcc

  Cloning https://github.com/bitbankinc/python-bitbankcc (to revision fba9f83) to /tmp/pip-install-oh4qqyv6/python-bitbankcc_3c2ab9bb8d92456a8de056bfb6e2d2e1
  Running command git clone --filter=blob:none --quiet https://github.com/bitbankinc/python-bitbankcc /tmp/pip-install-oh4qqyv6/python-bitbankcc_3c2ab9bb8d92456a8de056bfb6e2d2e1
  Running command git checkout -q fba9f83
  Resolved https://github.com/bitbankinc/python-bitbankcc to commit fba9f83
  Preparing metadata (setup.py) ... done
  Created wheel for python-bitbankcc: filename=python_bitbankcc-0.0.6-py3-none-any.whl size=9600 sha256=9d489e8f8b51dd7e19fd06a71a5a6cf813a9b4416649a6c0484b408a6fca175a
  Stored in directory: /tmp/pip-ephem-wheel-cache-d9oc8usm/wheels/ff/a3/af/ddd305054bec732f97767c19f78e7f8a26a8367f09a3819c49
Successfully built python-bitbankcc


In [2]:
import time
import logging
import python_bitbankcc

# ログレベルをINFOに設定
logging.basicConfig(level=logging.INFO)

# 初期設定
print("初期設定を行います。")
API_KEY = ''
API_SECRET = ''
pub = python_bitbankcc.public()
prv = python_bitbankcc.private(API_KEY, API_SECRET)

# 各種パラメータ
print("各種パラメータを設定します。")
order_lot = 0.0001
time_frame = 2
price_diff_percent = 1.0
exit_profit_percent = 0.001
exit_loss_percent = 0.001

# ポジション情報
current_position = 0
entry_price = 0

while True:
    print("高値・安値を計算します。")
    value = pub.get_candlestick('btc_jpy', '1min', '20230906')
    ohlcv = value['candlestick'][0]['ohlcv'][-time_frame:]
    high_prices = [float(x[2]) for x in ohlcv]
    low_prices = [float(x[3]) for x in ohlcv]
    high_max = max(high_prices)
    low_min = min(low_prices)
    print(f"計算された高値: {high_max}, 計算された安値: {low_min}")

    print("現在価格を取得します。")
    ticker = pub.get_ticker('btc_jpy')
    current_price = float(ticker['last'])
    print(f"取得された現在価格: {current_price}")

    print("JPY残高を取得します。")
    assets = prv.get_asset()
    jpy_balance = float(assets['assets'][0]['free_amount'])
    print(f"取得されたJPY残高: {jpy_balance}")

    print("エントリー条件をチェックします。")
    if jpy_balance >= 10000 and current_position == 0:
        price_diff = 1 - low_min / high_max
        if price_diff <= high_max * (price_diff_percent / 100) and current_price > high_max:
            print("エントリー条件が満たされました。マーケットオーダーでエントリーします。")
            prv.order('btc_jpy', None, order_lot, 'buy', 'market')
            current_position = order_lot
            entry_price = current_price
            logging.info("Entered the position.")

    print("イグジット条件をチェックします。")
    if current_position != 0:
        if current_price >= entry_price * (1 + exit_profit_percent / 100):
            print("利益確定条件が満たされました。ポジションをクローズします。")
            prv.order('btc_jpy', None, current_position, 'sell', 'market')
            current_position = 0
            logging.info("Exited the position with profit.")
        elif current_price <= entry_price * (1 - exit_loss_percent / 100):
            print("損切り条件が満たされました。ポジションをクローズします。")
            prv.order('btc_jpy', None, current_position, 'sell', 'market')
            current_position = 0
            logging.info("Exited the position with loss.")

    print("10秒待機します。")
    time.sleep(10)



初期設定を行います。
各種パラメータを設定します。
高値・安値を計算します。
計算された高値: 3811001.0, 計算された安値: 3811001.0
現在価格を取得します。
取得された現在価格: 4146999.0
JPY残高を取得します。
取得されたJPY残高: 180330.1627
エントリー条件をチェックします。
エントリー条件が満たされました。マーケットオーダーでエントリーします。
イグジット条件をチェックします。
10秒待機します。


KeyboardInterrupt: ignored